Lab 7 (Reddit)

This script connect to Reddit API and crawls posts based on search terms.

Ref: http://www.storybench.org/how-to-scrape-reddit-with-python/

In [2]:
import praw
import pandas as pd
import datetime as dt
from elasticsearch import Elasticsearch
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from operator import itemgetter
import math
import numpy as np
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='', \
                     username='', \
                     password='')

In [4]:
subreddit = reddit.subreddit('hackathon')

In [5]:
new_subreddit = subreddit.new()

In [6]:
topics_dict = { "title":[], \
                "score":[], \
                "id":[], "url":[], \
                "comms_num": [], \
                "created": [], \
                "body":[]}
                

In [7]:
for submission in subreddit.new(limit=1000):
    # get the details of the post
    topics_dict["title"].append(submission.title.lower())
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["body"].append(submission.selftext.lower())


In [8]:
def calculateBOW(wordset,l_doc):
  tf_diz = dict.fromkeys(wordset,0)
  for word in l_doc:
      tf_diz[word]=l_doc.count(word)
  return tf_diz

In [9]:
# combine title and body
keywordbags = [x + y for x, y in zip(topics_dict["title"], topics_dict["body"])]
# remove punctuation
keywordbags = [x.replace('.', '').replace(',', '').replace('?', '').replace('!', '').replace('(', '').replace(')', '').replace(';', '').replace(':', '') for x in keywordbags]
# split into words
keywordbags = [x.split() for x in keywordbags]
# strip all words
for each_list in keywordbags:
  for i in range(len(each_list)):
    each_list[i] = each_list[i].strip('[').strip('"').strip('{').strip(']').strip('}').strip("'")
    
# lowercase all words
keywordbags = [[y.lower() for y in x] for x in keywordbags]
# remove empty strings
keywordbags = [[y for y in x if y] for x in keywordbags]
# remove stop words
stopwordz = ['the', 'a', 'an', 'and', 'or', 'but', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'will', 'would', 'shall', 'should', 'can', 'could', 'may', 'might', 'must', 'to', 'of', 'in', 'for', 'on', 'at', 'by', 'from', 'with', 'about', 'as', 'into', 'like', 'through', 'after', 'over', 'between', 'out', 'against', 'during', 'without', 'before', 'under', 'around', 'down', 'off', 'above', 'below', 'up', 'into', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', '&', 'your', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than', 'us', '-', '--', '*', "get", "it's"]
keywordbags = [[x for x in y if x not in stopwordz] for y in keywordbags]

stop_words = set(stopwords.words('english'))
keywordbags = [[x for x in y if x not in stop_words] for y in keywordbags]

# remove words words that contains http or www
keywordbags = [[x for x in y if 'http' not in x] for y in keywordbags]
keywordbags = [[x for x in y if 'www' not in x] for y in keywordbags]

In [16]:
all_words = [item for sublist in keywordbags for item in sublist]
df = pd.DataFrame(all_words, columns=['word'])
df.to_csv('all_words.csv', index=False)
    

In [11]:
es = Elasticsearch("http://localhost:9200")

In [12]:
# put a new index with mappings "mappings": {
    # "properties": {
    #   "mytext":{
        # "type":"text",
        # "fielddata": true
es.indices.delete(index='reddit', ignore=[400, 404])
# create a new index with mappings
es.indices.create(index='reddit', body={
    "mappings": {
        "properties": {
            "keyword":{
                "type":"text",
                "fielddata": True
            }
        }
    }
})

# insert keywordbags into elasticsearch
elastic_id = 1
for i in range(len(keywordbags)):
    for j in range(len(keywordbags[i])):
        # create a dictionary for each document
        doc = {}
        doc['keyword'] = keywordbags[i][j]
        doc['created'] = dt.datetime.now()
        # insert into elasticsearch
        es.index(index='reddit', id=elastic_id, document=doc)
        elastic_id += 1

C:\Users\Andy\AppData\Local\Temp\ipykernel_12496\3378907868.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='reddit', ignore=[400, 404])


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000027DEB6CD3C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [ ]:
# remove reddit.tf-idf indices
es.indices.delete(index='reddit.tf_idfscore', ignore=[400, 404])